# Gruppenaufgabe: Normalisierung

### Inhalt

* Wiederholung: Definition funktionaler Abhängigkeiten
* Programm zur Identifikation einer funktionalen Abhängigkeit in einer Tabelle
* Erweiterung des Programms um unscharfe und zusammengesetzte funktionale Abhängigkeiten
* Überblick über Algorithmen zur Erkennung funktionaler Abhängigkeiten
* Anwendung auf Datensätze

### Vorbereitung

In [1]:
import pandas as pd
import psycopg2
import yaml
from itertools import combinations

Verbindung zur Datenbank herstellen:

In [2]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
conn = psycopg2.connect(database = config['gruppenaufgabe']['db'], 
                        user = config['gruppenaufgabe']['user'], 
                        password = config['gruppenaufgabe']['pass'])
cur = conn.cursor()

Nützliche Funktionen:

In [3]:
def sql(sql: str):
    return pd.read_sql(sql, conn)

In [4]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

Tabelle zum Testen erstellen:

In [7]:
def create():
    with conn:
        with conn.cursor() as cur:
            cur.execute('CREATE TABLE test (A varchar(2), B varchar(2), C varchar(2),D varchar(2))')
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a1', 'b1', 'c1', 'd2'])
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a2', 'b2', 'c1', 'd1'])
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a1', 'b1', 'c2', 'd1'])
            cur.execute('INSERT INTO test(A, B, C, D) values(%s, %s, %s, %s);', ['a3', 'b1', 'c1', 'd1'])

In [ ]:
create()

### Aufgabe 1

Schreiben Sie ein (zunächst einfach gehaltenes) Programm, welches
(möglichst effizient) auf Basis des Systemkatalogs einer PostgreSQL
Datenbank funktionale Abhängigkeiten zwischen einzelnen Attributen $(A
\rightarrow B)$ einer Relation erkennen kann und darauf basierend Vorschläge für
die Auslagerung von Tabellen macht. Untersuchen Sie die Laufzeiten
unterschiedlicher Implementierungsvarianten.

**Was sind funktionale Abhängigkeiten?**

**Definition:** B ist von A funktional abhängig $(A
\rightarrow B)$ genau dann, wenn in einer Relation $R$ zu jedem Wert in $A$ genau ein Wert von $B$ existiert.

Formal: $\forall x, y \in R: x.A = y.A \Longrightarrow x.B = y.B$

**Beispiel:** 
* In der Relation $Student (\underline{MatrNr}, \underline{LVID}, Bezeichnung, Dozent, Note)$ ist die Bezeichnung und der Dozent der Lehrveranstaltung von LVID funktional abhängig, d.h. es gilt $LVID \rightarrow Bezeichnung$ und $LVID \rightarrow Dozent$.

Beispieltabelle:

In [6]:
sql('SELECT * FROM test;')

,a,b,c,d
0,a1,b1,c1,d2
1,a2,b2,c1,d1
2,a1,b1,c2,d1
3,a3,b1,c1,d1


* In der Relation $Test (A, B, C, D)$ gilt $A \rightarrow B$.

**Mögliche SQL-Skripts, die eine funktionale Abhängigkeit identifizieren**

Überprüfung der funktionalen Abhängigkeit $A \rightarrow B$. Wird eine leere Liste zurückgegeben, ist $B$ von $A$ funktional abhängig.

In [29]:
cur.execute('SELECT * FROM test t1, test t2 WHERE t1.a = t2.a AND t1.b <> t2.b')
print(cur.fetchall())

[]


In [9]:
cur.execute('SELECT a FROM test GROUP BY a HAVING COUNT(DISTINCT b) > 1;')
print(cur.fetchall())

[]


Welches der beiden SQL-Statements ist effizienter?

- der `EXPLAIN` Befehl liefert den Ausführungsplan eines SQL-Statements
- die Ausgabe ist als eine Baumstruktur zu lesen (von unten nach oben)

In [9]:
print_full(sql('EXPLAIN SELECT * FROM test t1, test t2 WHERE t1.a = t2.a AND t1.b <> t2.b;'))

                                                           QUERY PLAN
0                        Hash Join  (cost=1.09..2.23 rows=2 width=24)
1                            Hash Cond: ((t1.a)::text = (t2.a)::text)
2                         Join Filter: ((t1.b)::text <> (t2.b)::text)
3          ->  Seq Scan on test t1  (cost=0.00..1.04 rows=4 width=12)
4                         ->  Hash  (cost=1.04..1.04 rows=4 width=12)
5          ->  Seq Scan on test t2  (cost=0.00..1.04 rows=4 width=12)


Ausführungsplan als Baumstruktur:

Hash Join<br>
├── Seq Scan<br>
└── Hash<br>
$\;\;\;\;\;\;$└── Seq Scan<br>

- der zweite Wert bei `cost` schätzt die Dauer für jeden Knoten (in willkürlichen Einheiten)
- die geschätzten Kosten des gesamten SQL-Statements betragen 2.23

In [10]:
print_full(sql('EXPLAIN SELECT a FROM test GROUP BY a HAVING COUNT(DISTINCT b) > 1;'))

                                                       QUERY PLAN
0                GroupAggregate  (cost=1.08..1.15 rows=1 width=3)
1                                                    Group Key: a
2                                 Filter: (count(DISTINCT b) > 1)
3                      ->  Sort  (cost=1.08..1.09 rows=4 width=6)
4                                                     Sort Key: a
5          ->  Seq Scan on test  (cost=0.00..1.04 rows=4 width=6)


- das zweite SQL-Statement ist effizienter (gesamte Kosten bei 1.15)
- um Laufzeiten (in Sekunden) zu untersuchen kann auch der Befehl `EXPLAIN ANALYZE` verwendet werden 

**Programm zur Identifikation von funktionalen Abhängigkeiten in einer Relation**

In [13]:
def func_dep(tname: str):
    
    # Speichere Spaltennamen von Tabelle in Liste
    cols_df = sql(f'''SELECT column_name FROM information_schema.columns WHERE table_name = '{tname}';''')
    cols = cols_df['column_name'].values.tolist()
    
    dep = []
    
    with conn:
        with conn.cursor() as cur:
            
            # Iteriere über alle Spaltenpaare
            for i in cols:
                for j in cols:
                    if i==j:
                        continue
                    cur.execute(f'SELECT {i} FROM {tname} GROUP BY {i} HAVING COUNT(DISTINCT {j}) > 1;')
                    res = cur.fetchall()
                    
                    # Falls die zurückgegebene Liste des SQL-Statements leer ist, wurde FA gefunden
                    if not res:
                        dep.append(f'{i} -> {j}')
                        
    return dep
                
func_dep('test')

['a -> b']

Komplexitätsklasse: $O(n^2 m^2)$ wobei $n$ die Anzahl der Spalten  und $m$ die Anzahl der Zeilen einer Relation ist

### Aufgabe 2

Versuchen Sie, Ihr Programm um die folgenden Features zu erweitern:

* Berücksichtigung “unscharfer” funktionaler Abhängigkeiten, $(A→ B \text{ zu } 98\%)$
* Berücksichtigung funktionaler Abhängigkeiten zwischen mehreren Attributen $(A → BC , AB→C, AB→CD)$

**Berücksichtigung unscharfer Abhängigkeiten**

In [65]:
def func_dep_unclear(tname: str):
    
    cols_df = sql(f'''SELECT column_name FROM information_schema.columns WHERE table_name = '{tname}';''')
    cols = cols_df['column_name'].values.tolist() 
    
    dep = {}
    
    with conn:
        with conn.cursor() as cur:
            for i in cols:
                for j in cols:
                    if i==j:
                        continue
                    
                    # SQL-Statement
                    cur.execute(f'''SELECT COUNT("{i}"),COUNT(DISTINCT "{j}") FROM {tname} GROUP BY "{i}";''')
                    res = cur.fetchall()
                    
                    # z.B. für A -> C:
                    # A    C
                    # a_1  c_1
                    # a_2  c_1
                    # a_1  c_2
                    # a_3  c_1
                    
                    # res = [(2,2), (1,1), (1,1)]
                    
                    n = 0
                    p = 0
                    
                    # Durchlaufe Tupel der Ergebnisliste
                    for k in range(0,len(res)):
                        
                        # Summiere über den ersten Eintrag im Tupel, d.h. n = Anzahl der Zeilen
                        n = n + res[k][0]
                        
                        # p = Anzahl der Zeilen, bei denen eine FA besteht
                        if res[k][1] == 1:
                            p = p + res[k][0]
                    p = p/n # p = 2, n = 4 
                    
                    # In 50% der Fälle ist C von A funktional abhängig
                    
                    # Speichere FA's in einem dictionary
                    if i not in dep.keys():
                        dep[i] = [(j,p)]
                    else:
                        dep[i].append((j,p))
                    
    return dep
                
func_dep_unclear('test')

{'a': [('b', 1.0), ('c', 0.5), ('d', 0.5)],
 'b': [('a', 0.25), ('c', 0.25), ('d', 0.25)],
 'c': [('a', 0.25), ('b', 0.25), ('d', 0.25)],
 'd': [('a', 0.25), ('b', 0.25), ('c', 0.25)]}

**Berücksichtigung funktionaler Abhängigkeiten zwischen mehreren Attributen**

In [66]:
def func_dep_unclear_all(tname: str):
    
    cols_df = sql(f'''SELECT column_name FROM information_schema.columns WHERE table_name = '{tname}';''')
    cols = cols_df['column_name'].values.tolist()
    
    dep = {}
    
    with conn:
        with conn.cursor() as cur:
            count = 0
            for i in range(0,len(cols)):
                for j in range(0,len(cols)):
                    if i==j:
                        continue
                        
                    # Prüfe auf FA der Form A -> B
                    cur.execute(f'''SELECT COUNT("{cols[i]}"),COUNT(DISTINCT "{cols[j]}") FROM {tname} 
                                    GROUP BY "{cols[i]}";''')
                    res = cur.fetchall()
                    
                    # Berechne unscharfe FA
                    n = 0
                    p = 0
                    for k in range(0,len(res)):
                        n = n + res[k][0]
                        if res[k][1] == 1:
                            p = p+ res[k][0]
                    p = p/n
                    
                    # Speichere FA's in einem dictionary
                    if cols[i] not in dep.keys():
                        dep[cols[i]] = [(cols[j],p)]
                    else:
                        dep[cols[i]].append((cols[j],p))
                
                # Vermeide doppelte Prüfung der FA. Z.B. ist AB -> C identisch zu BA -> C,
                # daher startet für i = Spalte B die zweite for-Schleife erst bei j = Spalte B
                for j in range(0 + count,len(cols)):
                    if i==j:
                        continue

                    for k in range(0,len(cols)):
                        if i==k or j==k:
                            continue

                        # Prüfe auf FA der Form AB -> C
                        cur.execute(f'''SELECT "{cols[i]}","{cols[j]}" FROM {tname} GROUP BY "{cols[i]}","{cols[j]}" 
                                        HAVING COUNT(DISTINCT "{cols[k]}") > 1;''')
                        res = cur.fetchall()
                        
                        if not res:                        
                            if f'{cols[i]}, {cols[j]}' not in dep.keys():
                                dep[f'{cols[i]}, {cols[j]}'] = [(cols[k],1)]
                            else:
                                dep[f'{cols[i]}, {cols[j]}'].append((cols[k],1))
                        
                count += 1
                
    return dep

                        
                        
                
dep = func_dep_unclear_all('test')
dep

{'a': [('b', 1.0), ('c', 0.5), ('d', 0.5)],
 'a, c': [('b', 1), ('d', 1)],
 'a, d': [('b', 1), ('c', 1)],
 'b': [('a', 0.25), ('c', 0.25), ('d', 0.25)],
 'c': [('a', 0.25), ('b', 0.25), ('d', 0.25)],
 'd': [('a', 0.25), ('b', 0.25), ('c', 0.25)]}

In [67]:
def parse_dep(dep):
    print('Funktionale Abhängigkeiten:')
    for key, values in dep.items():
        lst = [] 
        
        # FA der Form A -> B und AB -> C parsen
        for value in values:
            print(f'{key} -> {value[0]}: {value[1]}')
            if value[1]==1:
                lst.append(value[0])
        
        # FA der Form A -> BC und AB -> CD parsen
        # Die FA A -> BC ergibt sich aus den FA A -> B und A -> C
        # Die FA AB -> CD ergibt sich aus den FA AB -> C und AB -> D
        if len(lst) > 1:
            combs = combinations(lst, 2)
            for comb in list(combs):
                print(f'{key} -> {comb[0]}, {comb[1]}: 1')

parse_dep(dep)

Funktionale Abhängigkeiten:
a -> b: 1.0
a -> c: 0.5
a -> d: 0.5
a, c -> b: 1
a, c -> d: 1
a, c -> b, d: 1
a, d -> b: 1
a, d -> c: 1
a, d -> b, c: 1
b -> a: 0.25
b -> c: 0.25
b -> d: 0.25
c -> a: 0.25
c -> b: 0.25
c -> d: 0.25
d -> a: 0.25
d -> b: 0.25
d -> c: 0.25


Weitere Informationen: [Armstrong's axioms](https://en.wikipedia.org/wiki/Armstrong%27s_axioms)

### Aufgabe 3

Sobald Sie bei (2) nicht mehr unmittelbar weiterkommen: recherchieren
Sie, welche Ansätze für die o.g. Probleme bereits zur Verfügung stehen
(Paper, Implementierungen) und versuchen Sie zu verstehen, wie diese
grundsätzlich funktionieren. Vergleichen Sie diese mit Ihrem eigenen
Ansatz.

**Vorhandene Implementierung einer Postgresql Datenbank**

In [ ]:
cur.execute('CREATE STATISTICS s1 (dependencies) ON a,b,c,d FROM test') 
cur.execute('ANALYZE test')   

In [19]:
cur.execute('SELECT dependencies FROM pg_stats_ext')
print(cur.fetchall())

[('{"1 => 2": 1.000000, "1 => 3": 0.500000, "1 => 4": 0.500000, "2 => 1": 0.250000, "2 => 3": 0.250000, "2 => 4": 0.250000, "3 => 1": 0.250000, "3 => 2": 0.250000, "3 => 4": 0.250000, "4 => 1": 0.250000, "4 => 2": 0.250000, "4 => 3": 0.250000, "1, 2 => 3": 0.500000, "1, 2 => 4": 0.500000, "1, 3 => 2": 1.000000, "1, 3 => 4": 1.000000, "1, 4 => 2": 1.000000, "1, 4 => 3": 1.000000, "2, 3 => 1": 0.500000, "2, 3 => 4": 0.500000, "2, 4 => 1": 0.500000, "2, 4 => 3": 0.500000, "3, 4 => 1": 0.500000, "3, 4 => 2": 0.500000, "1, 2, 3 => 4": 1.000000, "1, 2, 4 => 3": 1.000000, "1, 3, 4 => 2": 1.000000, "2, 3, 4 => 1": 1.000000}',)]


### Aufgabe 4

Testen Sie Ihren Ansatz:
* Mit den als “flat table” geladenen NHTSA Complaints
* Mit einem beliebigen anderen für Sie interessanten Datensatz

**NHTSA Complaints**

In [21]:
sql('''SELECT * FROM complaint LIMIT 5;''')

,CMPLID,ODINO,MFR_NAME,MAKETXT,MODELTXT,YEARTXT,CRASH,FAILDATE,FIRE,INJURED,...,RESTRAINT_TYPE,DEALER_NAME,DEALER_TEL,DEALER_CITY,DEALER_STATE,DEALER_ZIP,PROD_TYPE,REPAIRED_YN,MEDICAL_ATTN,VEHICLES_TOWED_
0,1,958241,"Volvo Car USA, LLC",VOLVO,760,1987.0,N,NaN,N,0,...,None,None,None,None,None,None,V,N,N,N
1,2,958130,Ford Motor Company,FORD,THUNDERBIRD,1992.0,N,19941222.0,N,0,...,None,None,None,None,None,None,V,N,N,N
2,3,958132,Kia Motors America,KIA,SEPHIA,1994.0,Y,19941230.0,N,0,...,None,None,None,None,None,None,V,N,N,N
3,4,958133,"Chrysler (FCA US, LLC)",DODGE,600,1987.0,N,19941231.0,N,0,...,None,None,None,None,None,None,V,N,N,N
4,5,958137,"Chrysler (FCA US, LLC)",DODGE,CARAVAN,1991.0,N,19941218.0,N,0,...,None,None,None,None,None,None,V,N,N,N


In [22]:
sql('''SELECT count(*) FROM complaint;''')

,count
0,882281


In [9]:
func_dep_unclear('complaint')

{'CMPLID': [('ODINO', 1.0),
  ('MFR_NAME', 0.9999943328712735),
  ('MAKETXT', 0.9999943328712735),
  ('MODELTXT', 0.9999943328712735),
  ('YEARTXT', 0.9999943328712735),
  ('CRASH', 1.0),
  ('FAILDATE', 0.87584114358124),
  ('FIRE', 1.0),
  ('INJURED', 1.0),
  ('DEATHS', 1.0),
  ('COMPDESC', 0.9998605886333266),
  ('CITY', 0.9999648638018953),
  ('STATE', 0.9999886657425469),
  ('VIN', 0.7655701528197932),
  ('DATEA', 1.0),
  ('LDATE', 1.0),
  ('MILES', 0.42486577405611137),
  ('OCCURENCES', 0.7643415193118746),
  ('CDESCR', 0.99996373037615),
  ('CMPL_TYPE', 0.9999988665742547),
  ('POLICE_RPT_YN', 0.9999988665742547),
  ('PURCH_DT', 0.39853856084399414),
  ('ORIG_OWNER_YN', 0.9999988665742547),
  ('ANTI_BRAKES_YN', 0.9999988665742547),
  ('CRUISE_CONT_YN', 0.9999988665742547),
  ('NUM_CYLS', 0.2768585065302324),
  ('DRIVE_TRAIN', 0.441863759958562),
  ('FUEL_SYS', 0.3602514391673401),
  ('FUEL_TYPE', 0.46645683178035113),
  ('TRANS_TYPE', 0.3595475817795011),
  ('VEH_SPEED', 0.442251

**Business-Tabelle**

In [63]:
sql('select * from business limit 5;')

,is_open,business_id,id_third,id_second,id_first,name,address,city,state,postal,latitude,longitude,stars,review_count,distance_first,distance_second,distance_third,categories
0,0,--164t1nclzzmca7eDiJMw,USC00410431,US1TXTV0198,USC00410420,Me So Hungry,1104 E 6th St,Austin,TX,78702,30.264896,-97.731028,4.0,137,0.373439,0.572124,0.632392,None
1,0,--6COJIAjkQwSUZci_4PJQ,US1ORMT0049,US1ORMT0128,US1ORMT0024,Medley,7881 SW Capitol Hwy,Portland,OR,97219,45.467868,-122.714524,4.0,99,0.229855,0.701664,0.705749,None
2,1,--Q3mAcX9t63f7Xcbn7LVA,US1OHDL0017,US1OHFR0024,US1OHDL0012,The Royce,8791 Lyra Dr,Columbus,OH,43240,40.145557,-82.977284,4.5,107,0.979585,2.240159,2.982919,None
3,1,--UNNdnHRhsyFUbDgumdtQ,US1ORMT0033,US1ORMT0132,USW00024274,Le Pigeon,738 E Burnside St,Portland,OR,97214,45.522796,-122.657872,4.5,1236,1.165859,1.785909,2.274014,None
4,0,--_nBudPOb1lNRgKfjLtrw,US1OHFR0106,US1OHFR0002,US1OHFR0016,Mezcal Cantina & Grill,1993 Hard Rd,Columbus,OH,43235,40.117275,-83.068481,4.0,8,0.558175,0.657653,0.840526,None


In [58]:
dep = func_dep_unclear('business')

['latitude', 'longitude', 'stars', 'review_count', 'distance_first', 'distance_second', 'distance_third', 'is_open', 'state', 'postal', 'business_id', 'id_third', 'id_second', 'id_first', 'name', 'address', 'city']


In [62]:
parse_dep(dep)

Funktionale Abhängigkeiten:
latitude -> longitude: 0.9285911167112931
latitude -> stars: 0.8345999369979524
latitude -> review_count: 0.802035753661994
latitude -> distance_first: 0.9191014332965821
latitude -> distance_second: 0.9191014332965821
latitude -> distance_third: 0.9191014332965821
latitude -> is_open: 0.9044928335170893
latitude -> state: 0.9991730981256891
latitude -> postal: 0.9434359741691605
latitude -> business_id: 0.8001063159552686
latitude -> id_third: 0.9597771302567334
latitude -> id_second: 0.960308710033076
latitude -> id_first: 0.9625728461174988
latitude -> name: 0.8008938415498503
latitude -> address: 0.8777169633013073
latitude -> city: 0.9784414868483226
longitude -> latitude: 0.9226649866120649
longitude -> stars: 0.8319814143959678
longitude -> review_count: 0.799436919199874
longitude -> distance_first: 0.9136084422743739
longitude -> distance_second: 0.9136084422743739
longitude -> distance_third: 0.9136084422743739
longitude -> is_open: 0.9069144747204